In [199]:
import numpy as np
import re
import math
import sys
import string
from copy import deepcopy
import itertools
MAX_INT16 = np.iinfo(np.int16).max;
MAX_INT32 = np.iinfo(np.int32).max;
from IPython.display import display, clear_output
np.set_printoptions(suppress=True,linewidth=np.nan,threshold=sys.maxsize)

In [200]:
f = open("../input/day12.txt","r")
lines = f.read().splitlines()
lines_iter = iter(lines)
len(lines)

41

In [201]:
# row col
grid = np.array([[string.ascii_letters.index(r) for r in l] for l in lines])

In [202]:
def np_where_2d(comparison):
     return list(zip(*np.where(comparison)))

In [203]:
def flattern_point(point, grid):
    return len(grid[0]) * point[0] + point[1]

def deflattern_point(point_f, grid):
    a = int(point_f / len(grid[0]))
    b = point_f % len(grid[0]) 
    return (a,b)

In [205]:
start = np_where_2d(grid == string.ascii_letters.index("S"))[0]
grid[start] = string.ascii_letters.index("a")
end = np_where_2d(grid == string.ascii_letters.index("E"))[0]
grid[end] = string.ascii_letters.index("z")

end_f = flattern_point(end, grid)
start_f = flattern_point(start, grid)

print(start, grid[start], end, grid[end])

(20, 0) 0 (20, 120) 25


In [206]:
def compute_heuristic_map(input_map):
    rows = len(input_map)
    columns = len(input_map[0])
    heuristics = np.zeros((rows, columns), np.int32)
    for r in range(rows):
        for c in range(columns):
            # heuristics[r][c] = rows -r + columns - c -2
            heuristics[r][c] = abs(end[0] - r) + abs(end[1] - c)
    return heuristics

In [207]:
def list_contains(l,el):
    return len(l[l == el]) > 0

In [208]:
def a_star_climbing(levels_map, start, end):
    levels_map_flat = levels_map.flatten()
    heuristic_map = compute_heuristic_map(levels_map)
    heuristic_flat_map = heuristic_map.flatten()
    
    row_length = len(levels_map[0])
    col_length = len(levels_map)

    end_f = flattern_point(end, grid)
    start_f = flattern_point(start, grid)

    # f-value, startpoint flat
    openlist = [(0, start_f)] 
        
    g_values_flat = np.full(col_length*row_length, 0, np.int32)
    g_values_flat[0] = 0
    previous_flat_map = np.full(col_length*row_length, -1, np.int32)
    previous_flat_map[0] = 0
    closedlist  = np.array([], np.int32)

    
    # überprüft alle Nachfolgeknoten und fügt sie der Open List hinzu, wenn entweder
    # - der Nachfolgeknoten zum ersten Mal gefunden wird, oder
    # - ein besserer Weg zu diesem Knoten gefunden wird
    def expand_node(current_node):
        nonlocal openlist
        nonlocal closedlist
        successors = np.array([], np.int32);
        curr_row = int(current_node/row_length)
        curr_col = current_node%row_length
        if curr_row != 0:
            successors = np.append(successors, (curr_row - 1) * row_length + curr_col)
        if curr_row != len(levels_map)-1:
            successors = np.append(successors, (curr_row + 1)  * row_length + curr_col)     
        if curr_col != 0:
            successors = np.append(successors, curr_row * row_length + curr_col -1)
        if curr_col != len(levels_map[0]) - 1:
            successors = np.append(successors, curr_row * row_length + curr_col + 1)       

        for successor in successors:
            # wenn der Nachfolgeknoten bereits auf der Closed List ist – tue nichts
            if list_contains(closedlist, successor):
                continue

            # advnet of code day 12: climb only on next high
            if (levels_map_flat[successor] - levels_map_flat[current_node]) > 1:
                continue

            # g-Wert für den neuen Weg berechnen: g-Wert des Vorgängers plus
            # die Kosten der gerade benutzten Kante
            tentative_g = g_values_flat[current_node] + levels_map_flat[current_node]
            
            # wenn der Nachfolgeknoten bereits auf der Open List ist,
            # aber der neue Weg nicht besser ist als der alte – tue nichts
            openlist_succ_index = [i for i, x in enumerate(openlist) if x[1] == successor]

            if len(openlist_succ_index) and tentative_g >= g_values_flat[successor]:
                continue
    
            # Vorgängerzeiger setzen und g Wert merken oder anpassen
            previous_flat_map[successor] = current_node
            g_values_flat[successor] = tentative_g
            
            # f-Wert des Knotens in der Open List aktualisieren
                
            # bzw. Knoten mit f-Wert in die Open List einfügen
            f = tentative_g + heuristic_flat_map[successor]

            if not len(openlist_succ_index): 
                openlist.append((f, successor))
                openlist.sort(reverse=True)
            else:
                openlist[openlist_succ_index[0]] = (f, successor)

            
    # diese Schleife wird durchlaufen bis entweder
    # - die optimale Lösung gefunden wurde oder
    # - feststeht, dass keine Lösung existiert
    while len(openlist) != 0:        
        # Knoten mit dem geringsten f-Wert aus der Open List entfernen
        current_node_f = openlist.pop()
        
        # Wurde das Ziel gefunden?
        if current_node_f[1] == end_f:
            current_node = deflattern_point(current_node_f[1], grid)
            return True, previous_flat_map, current_node
        
        # Der aktuelle Knoten soll durch nachfolgende Funktionen
        # nicht weiter untersucht werden, damit keine Zyklen entstehen
        closedlist = np.append(closedlist, current_node_f[1])
        
        # Wenn das Ziel noch nicht gefunden wurde: Nachfolgeknoten
        # des aktuellen Knotens auf die Open List setzen
        expand_node(current_node_f[1])
    
    # die Open List ist leer, es existiert kein Pfad zum Ziel
    current_node = deflattern_point(current_node_f[1], grid)
    return False, previous_flat_map, current_node

In [209]:
def mark_previous(p_map, start, curr):
    while True:
        if curr == start:
            return p_map
        previous = p_map[curr]
        p_map[curr] = -9
        curr = previous

In [210]:
def count_previous(p_map, start, curr, levels):
    all_sum = 0
    while True:
        if curr == start:
            return all_sum
        previous = p_map[curr]
        all_sum += 1
        curr = previous

In [211]:
def print_a_star_info(levels, p_map, start, goal, print_info=False):
    goal_f = len(levels[0]) * goal[0] + goal[1]
    start_f = len(levels[0]) * start[0] + start[1]
    best_way = count_previous(p_map.copy(), start_f, goal_f, levels.flatten())

    if print_info:
        print("\nbest way: ", best_way) 
        print("visited: ", len(p_map[p_map != -1]))
        print("not visited: ", len(p_map[p_map == -1]))
        print(mark_previous(p_map.copy(), start_f, goal_f).reshape((len(levels),len(levels[0]))))
    return mark_previous(p_map.copy(), start_f, goal_f).reshape((len(levels),len(levels[0])))

In [212]:
success, previous_map_test, last_node = a_star_climbing(grid, start, end)
if not success:
    print("no solution found")
# _ = print_a_star_info(grid, previous_map_test, start, last_node, True)
count_previous(previous_map_test.copy(), start_f, end_f, grid.flatten())

468

In [216]:
all_a_pos = np_where_2d(grid == string.ascii_letters.index("a"))
print(len(all_a_pos))

all_a_counts = []

for pos in all_a_pos:
    success, previous_map_test, last_node = a_star_climbing(grid, pos, end)
    if success:
        all_a_counts.append(count_previous(previous_map_test.copy(), flattern_point(pos, grid), end_f, grid.flatten()))


1850


In [217]:
min(all_a_counts)

459